In [10]:
# !pip3.10 install  langchain_community langchain_experimental langchain_ollama unstructured pdf2image pdfminer open-clip-torch pylzma langchain faiss-cpu PyPDF2
# !pip3.10 install transformers -U

# !ollama pull llava-phi3
# !ollama pull llama3.2-vision 
# !ollama pull moondream
# !ollama pull llama3.2:1b
# !ollama pull llama3.2
# !ollama pull granite3.1-moe:3b
# !ollama pull smollm2:1.7b
# !ollama pull smollm2:1.7b-instruct-fp16

In [11]:
from ollama import chat
# from ollama import ChatResponse
from tqdm import tqdm
import textwrap
import time
import pandas as pd


def ask_llm(model, question, context = None, image_path_list = None):
    res = chat(
	model=model,
	messages=[{'role': 'user',
            'content': context + question}]
    )
    
    output_ = res['message']['content']
    # return textwrap.fill(output_, width=150)
    return output_

In [12]:
# reading the prompts

prompt_list = pd.read_csv("/Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/Phi_LLM/data_local/documents/100_prompts_2.csv", index_col = 0)
prompt_list.head()

,Topic,Human-like Prompt,filename,index
66,Array Formulas,Sum product but not regular.,Niche_Unclear_Excel_Prompts,0
43,Automation and Macros,Can I assign a macro to a button for quick exe...,Excel_Help_Prompts,1


# Random ask

In [13]:
context = '''
You are an Microsoft Excel instructor who provides crisp, clear and concise instructions to user prompts.
All your responses are 500 words or less.
User should be able to action on the instructions and solve their problems.
Follow the below format for your response.

[START]
Reason: a single sentence explanation of all below steps with reasoning.
Step: mouse or keyboard actions user has to perform.
Step: mouse or keyboard actions user has to perform.
...
[END]

'''

model_list = ["llava-phi3", 
            #   "llama3.2-vision",
              "moondream",
              "llama3.2:1b", 
              "llama3.2",
              "granite3.1-moe:3b",
            #   "smollm2:1.7b",
              "smollm2:1.7b-instruct-fp16"
              ]

# Create a dictionary to store results
results = {
    "index_":prompt_list.index.to_list(),
    "topics": prompt_list['Topic'].tolist(),
    "prompt": prompt_list['Human-like Prompt'].tolist(), }

# Initialize dictionary to store model outputs and times
for model_ in model_list:
    results[model_] = []          # For model output
    results[f"{model_}_time"] = []  # For model execution time

# Iterate over prompts and models
for prompt in tqdm(prompt_list['Human-like Prompt'].tolist(), desc="Processing Prompts"):
    for model_ in tqdm(model_list, leave=False, desc="Processing Models"):
        try:
            start_time = time.time()  # Start timing
            output = ask_llm(model=model_, question=prompt, context=context)
            end_time = time.time()  # End timing

            elapsed_time = end_time - start_time  # Calculate time taken

            # Append results for the current model and prompt
            results[model_].append(output)
            results[f"{model_}_time"].append(f"{elapsed_time:.2f} seconds")
        except Exception as e:
            # Handle any errors during model execution
            print(f"Error with model '{model_}' and prompt '{prompt}': {e}")
            results[model_].append("Error")
            results[f"{model_}_time"].append("Error")

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
csv_path = "/Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/Phi_LLM/data_local/outputs/model_outputs_multiple_prompts.csv"
df.to_csv(csv_path, index=False)

print(f"Results saved to {csv_path}")

Processing Prompts: 100%|██████████| 2/2 [01:19<00:00, 39.89s/it]

Results saved to /Users/lekshmanannatarajan/Library/CloudStorage/GoogleDrive-zuari1993@gmail.com/My Drive/Colab Notebooks/Phi_LLM/data_local/outputs/model_outputs_multiple_prompts.csv
